In [65]:
import stormpy
import stormpy.simulator

def build_trans_matrix(s):

    builder = stormpy.SparseMatrixBuilder(rows=0, columns=0, entries=0, force_dimensions=False,
                                              has_custom_row_grouping=True, row_groups=0)

    #  s = maximale aantal stappen in de mdp
    row_counter = 0
    for x in range(s):
        print("CURRENT LAYER:", x)

        #transitions from state 0
        builder.new_row_group(row_counter)
        builder.add_next_value(row_counter, 1 + (x+1) * 2, 1)
        print("TRANSITION FROM {} TO {}".format(row_counter, 1 + 2*x + 2))
        row_counter += 1
        builder.add_next_value(row_counter, 0 + (x+1) * 2, 1)
        print("TRANSITION FROM {} TO {}".format(row_counter - 1, 0 + 2*x + 2))
        row_counter += 1
        
        #Go to state 1 with probability 1

        #transitions from state 1
        builder.new_row_group(row_counter)
        builder.add_next_value(row_counter, 1 + (x+1) * 2, 1)
        print("TRANSITION FROM {} TO {}".format(row_counter, 1 + 2*x + 2))
        row_counter += 1
        #Stay in this state
        
        if x == s - 1:
            print("MAKING STATES ABSORBING")
            #make all final states absorbing
            builder.new_row_group(row_counter)
            builder.add_next_value(row_counter, 0 + (x+1) * 2, 1)
            row_counter += 1
            
            builder.new_row_group(row_counter)
            builder.add_next_value(row_counter, 1 + (x+1) * 2, 1)
            row_counter += 1
    
    transition_matrix = builder.build()
    
    return transition_matrix

def build_reward_matrix(s):
    builder = stormpy.SparseMatrixBuilder(rows=0, columns=0, entries=0, force_dimensions=False,
                                              has_custom_row_grouping=True, row_groups=0)

    # s = maximale aantal stappen in de mdp
    row_counter = 0
    for x in range(s):
        #transitions from state 0
        builder.new_row_group(row_counter)
        builder.add_next_value(row_counter, 1 + (x+1) * 2, 2)
        row_counter += 1
        builder.add_next_value(row_counter, 0 + (x+1) * 2, 2)
        row_counter += 1
        
        #transitions from state 1
        builder.new_row_group(row_counter)
        builder.add_next_value(row_counter, 1 + (x+1) * 2, 1)
        row_counter += 1
        #Stay in this state 
        
        if x == s - 1:
            #make all final states absorbing
            builder.new_row_group(row_counter)
            builder.add_next_value(row_counter, 0 + (x+1) * 2, 0)
            row_counter += 1
            
            builder.new_row_group(row_counter)
            builder.add_next_value(row_counter, 1 + (x+1) * 2, 0)
            row_counter += 1

    
    transition_matrix = builder.build()
    
    return transition_matrix



def run_exp():
    s = 1 # Max amount of steps to be taken before rewards are set to 0
    transition_matrix = build_trans_matrix(s)
    #print(transition_matrix)
    transition_rewards = build_reward_matrix(s)
    #print(transition_rewards)
    
    reward_models = {}
    reward_models['simple'] = stormpy.SparseRewardModel(optional_transition_reward_matrix=transition_rewards)

    state_labeling = stormpy.storage.StateLabeling(2 + s*2)
    # Add labels
    labels = {'done', 'init', '0', '1'}
    for label in labels:
        state_labeling.add_label(label)
    
    state_labeling.add_label_to_state('init', 0)
    
    for layer in range(s + 1):
        
        state_labeling.add_label_to_state('0', 0 + (layer) * 2)
        state_labeling.add_label_to_state('1', 1 + (layer) * 2)
    
    # Collect components
    components = stormpy.SparseModelComponents(transition_matrix=transition_matrix, state_labeling = state_labeling,
                                                   reward_models=reward_models, rate_transitions=False)
    
    
    mdp = stormpy.storage.SparseMdp(components)
    print(mdp)
    
    for state in mdp.states:
        if state.id in mdp.initial_states:
            print("State {} is initial".format(state.id))
        for action in state.actions:
            for transition in action.transitions:
                print("From state {} by action {}, go to state {}".format(state, action, transition.column))

    
    prop = "Rmax=? [C]" #cumulative reward
    properties = stormpy.parse_properties(prop)
    result = stormpy.model_checking(mdp, properties[0], only_initial_states = True, extract_scheduler=True)
    
    assert result.has_scheduler
    scheduler = result.scheduler
    print(result.at(0))
    
    #try to simulate scheduler:
    
    current_state = 0 # Maak hier L.state_dict[(0,0)] van, of andere initial state. 
    visited_terrains = []
    simulator = stormpy.simulator.create_simulator(mdp, seed=42)
    
    for step in range(s):
        choice = scheduler.get_choice(current_state)
        action = choice.get_deterministic_choice()
        
        new_state, reward, labels = simulator.step(action)
        visited_terrains.append(list(labels)[0])
        
        #print("new state", new_state)
        #print(reward)
        current_state = new_state
    
    #print(visited_terrains)
    return result.scheduler
print(run_exp())


CURRENT LAYER: 0
TRANSITION FROM 0 TO 3
TRANSITION FROM 0 TO 2
TRANSITION FROM 2 TO 3
MAKING STATES ABSORBING
-------------------------------------------------------------- 
Model type: 	MDP (sparse)
States: 	4
Transitions: 	5
Choices: 	5
Reward Models:  simple
State Labels: 	4 labels
   * 0 -> 2 item(s)
   * init -> 1 item(s)
   * done -> 0 item(s)
   * 1 -> 2 item(s)
Choice Labels: 	none
-------------------------------------------------------------- 

State 0 is initial
From state 0 by action 0, go to state 3
From state 0 by action 1, go to state 2
From state 1 by action 0, go to state 3
From state 2 by action 0, go to state 2
From state 3 by action 0, go to state 3
2.0
___________________________________________________________________
Fully defined memoryless deterministic scheduler:
model state:    choice(s)
           0    0
           1    0
           2    0
           3    0
___________________________________________________________________

